## Crawling Berita

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL dasar situs berita Detik
base_url = "https://news.detik.com"

def get_news_detail(link):
    """Ambil isi berita dari halaman detail"""
    try:
        res = requests.get(link, timeout=10)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, 'html.parser')

        # Ambil isi paragraf berita
        body = soup.find('div', class_='detail__body-text itp_bodycontent')
        if not body:
            return "", ""
        paragraphs = body.find_all('p')
        content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # Ambil kategori berita (nama rubrik)
        breadcrumb = soup.find('div', class_='breadcrumb')
        category = breadcrumb.find_all('a')[1].text.strip() if breadcrumb and len(breadcrumb.find_all('a')) > 1 else "Tidak diketahui"

        return content, category
    except Exception as e:
        print(f"Gagal ambil detail: {link}, error: {e}")
        return "", "Tidak diketahui"

def scrape_detik(limit=50):
    """Scraping berita Detik dengan batas tertentu"""
    all_news = []
    page = 1
    id_berita = 1

    while len(all_news) < limit:
        url = f"{base_url}/indeks?page={page}"
        print(f"Scraping halaman {page}...")
        try:
            r = requests.get(url, timeout=10)
            r.raise_for_status()
        except Exception as e:
            print(f"Gagal ambil halaman {page}: {e}")
            break

        soup = BeautifulSoup(r.text, 'html.parser')
        articles = soup.find_all('a', class_='media__link')

        if not articles:
            print("Tidak ada artikel ditemukan, berhenti.")
            break

        for a in articles:
            if len(all_news) >= limit:
                break

            link = a.get('href')
            title = a.get_text(strip=True)

            if link and link.startswith('/'):
                link = base_url + link

            if not link or not title:
                continue

            isi, kategori = get_news_detail(link)
            if not isi:
                continue

            all_news.append({
                'idberita': id_berita,
                'judul_berita': title,
                'isi_berita': isi,
                'kategori_berita': kategori,
                'url': link
            })
            print(f"[{id_berita}] {title}")
            id_berita += 1
            time.sleep(1)  # Hindari pemblokiran server

        page += 1

    return all_news

# Jalankan scraping
news_data = scrape_detik(limit=100)

# Simpan ke CSV
df = pd.DataFrame(news_data)
df.to_csv("berita_detik.csv", index=False, encoding="utf-8-sig")
print("Berhasil menyimpan ke berita_detik.csv")


Scraping halaman 1...


Gagal ambil halaman 1: HTTPSConnectionPool(host='news.detik.com', port=443): Read timed out.
Berhasil menyimpan ke berita_detik.csv


In [2]:
# === BAGIAN 2: Membaca & Menampilkan CSV ===
import pandas as pd

df = pd.read_csv("berita_detik.csv")
print("=== Contoh Data Hasil Crawling ===")
print(df.head(10))  # tampilkan 10 berita pertama

# Jika dijalankan di Jupyter Notebook, bisa menampilkan tabel interaktif:
df.head()


EmptyDataError: No columns to parse from file